# 統計LOD NOTEBOOK

手探りでSPARQLを書きながら、統計LODの利用イメージを想像できたらと考えています。

## プロパティパスを使ったSPQRQL、使わないSPARQL

SPARQL初心者の自分は、プロパティパスを使った場合エリアコードをキーに興味のある様々な値を複数のプロパティを使ってどのように取得すべきかわからないので、冗長ではあるけどエリアコードの変数を明示してプロパティパスを使わずサンプルを書いています。

In [1]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display raw
# 新宿区の日本人の人口を求める（プロパティパス）
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja ;
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: raw

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

In [2]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display raw
# プロパティパスを使わないで書いてみる
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode ?areacode.
      ?areacode  ic:市区町村  "新宿区"@ja.
      ?s cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: raw

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

# 統計LODサポートを使って興味のあるプロパティを検索し、統計LODのデータを取得する

## 保育園数や保育園入所待機児童数を取得してみます。
- [統計LODサポート](http://idease.info/lodstat/)に「保育」と入力して検索。
- 「保育所数」「保育所在所児数」「保育所入所待機児童数」などのプロパティが表示されます。この中からまず、保育所入所待機児童数を取得することにします。
- 統計LODサポートで検索された「保育所入所待機児童数」をクリックします。
- 保育所数のプロパティ（predicate）「http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/J250502」 が長いのでPREFIXを"ssds-measure-2016"のように設定します。
- 待機児童数（この場合RDFの目的語になる）を指す変数を「?taikijido」とします
- セルID　 保育所数の述語　　待機児童数; のRDFを記述
- セルID（省略形式）　エリアコード（PREFIXを利用）を指す述語 エリアコード（未知の変数）. を記述
- [共通語彙基盤](http://imi.go.jp/ns/core/Core232.html)のコア語彙「市区町村」を使って、エリアコード  市区町村 新宿区. を記述し、新宿区のエリアコードを取得します。
- SPARQLを実行すると新宿区の待機児童数が変数taikijidoとして取得されます。


In [3]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
#新宿区の待機児童数
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:市区町村  "新宿区"@ja.
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['taikijido']},
 'results': {'bindings': [{'taikijido': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                         'type': 'typed-literal',
                                         'value': '176'}}]}}

## UNIONを使って「新宿区」と「文京区」のOR条件で取得する

In [4]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?保育所数
where{
        {
        ?s ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja. 
            } union {
        ?s ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode / ic:市区町村 "文京区"@ja. 
        }
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

保育所数
36
28


## 都道府県名のプロパティで東京都のエリアコードを取得し、東京都の待機児童数を取得

In [5]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?待機児童数 ?市区町村
where{
    ?s ssds-measure-2016:J250502 ?待機児童数;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?市区町村.
}
ORDER BY DESC(?待機児童数)
LIMIT 10

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

待機児童数,市区町村
5279,特別区部
884,世田谷区
578,練馬区
438,大田区
417,板橋区
416,江東区
294,足立区
285,杉並区
270,豊島区
257,町田市


## 区ごとの「保育所在所児童数」「保育所数」を組み合わせてみます。
プロパティパスで逆パス等を使うとまとめて記述できそうだけど、、、、自分は書けないのでとりあえずシンプルに書いてみます。

In [6]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?市区町村 ?保育所数  ?在所児童 ?待機児童
where{
    ?s1 ssds-measure-2016:J250502 ?待機児童;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?市区町村.
    ?s2 ssds-measure-2016:J2503 ?保育所数;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?在所児童;
        cd-dimension:standardAreaCode ?areacode.
}
ORDER BY DESC(?待機児童数)
LIMIT 20

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

市区町村,保育所数,在所児童,待機児童
目黒区,26,2498,132
文京区,28,2327,96
新島村,3,63,0
八丈町,3,162,0
東大和市,15,1788,79
小金井市,13,1404,188
清瀬市,11,937,52
国立市,11,1151,32
渋谷区,29,2305,73
あきる野市,15,1779,35


さらに関係ありそうなデータをエリアコードをキーに追加していきたいと思います。
統計LODサポートで今度は「病院」を検索します。「一般病院数」のプロパティがあるのでこれをSPARQLに追加します。
また最初のサンプルで取得しているような、区毎の人口もざっくり（全年齢、全盛別の値で）追加してみます。

In [7]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?siku ?hoikusyo  ?zaisyo ?taiki ?hospital ?jinko
where{
    ?s1 ssds-measure-2016:J250502 ?taiki;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?siku.
    ?s2 ssds-measure-2016:J2503 ?hoikusyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?zaisyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s4 ssds-measure-2016:I510120 ?hospital;
        cd-dimension:standardAreaCode ?areacode.
?s5 pc-measure-2010:population ?jinko;
        cd-dimension:standardAreaCode ?areacode;
        cd-dimension-2016:sex cd-code-2016:sex-total; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese; 
         pc-dimension-2010:area pc-code-2010:area-all; 
         cd-dimension-2016:age cd-code-2016:age-total .
}
ORDER BY DESC(?taiki)
LIMIT 20

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

siku,hoikusyo,zaisyo,taiki,hospital,jinko
特別区部,1158,115921,5279,407,8500358
世田谷区,82,8912,884,25,860559
練馬区,89,8814,578,17,694121
大田区,84,8756,438,27,666330
板橋区,93,8477,417,41,495700
江東区,72,7002,416,17,444062
足立区,88,8847,294,47,656755
杉並区,55,5390,285,17,506982
豊島区,31,3041,270,15,261282
町田市,56,5257,257,14,419062


保育所待機児童数でソートして表示してみました。市区町村の特別区部とは一般的には23区を指すようですが、市区町村とは別にデータが有るのがちょっと気になります。

現状（2016-12-30）だと、エンドポイントのレスポンスがこのぐらいのクエリになるとかなり時間がかかります。

### 集計したセルを含めてみる
本来データを取得してから、別のプロセスで集計を行うべきだと思うのですが、SPARQLで簡単な四則演算を反映した値を取得することもできます。試しに市区町村の人口に対する保育所数の比率を変数として取得してみます。

In [8]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
select ?siku ?hoikusyo  ?zaisyo ?taiki ?hospital ?jinko ?rate
where{
    ?s1 ssds-measure-2016:J250502 ?taiki;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?siku.
    ?s2 ssds-measure-2016:J2503 ?hoikusyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?zaisyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s4 ssds-measure-2016:I510120 ?hospital;
        cd-dimension:standardAreaCode ?areacode.
    ?s5 pc-measure-2010:population ?jinko;
        cd-dimension:standardAreaCode ?areacode;
        cd-dimension-2016:sex cd-code-2016:sex-total; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese; 
         pc-dimension-2010:area pc-code-2010:area-all; 
         cd-dimension-2016:age cd-code-2016:age-total .
    BIND (xsd:int(?hoikusyo)/xsd:int(?jinko) as ?rate)
}
ORDER BY DESC(?taiki)
LIMIT 20

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

siku,hoikusyo,zaisyo,taiki,hospital,jinko,rate
特別区部,1158,115921,5279,407,8500358,.000136229556449269548411961002113087472316
世田谷区,82,8912,884,25,860559,.00009528690072383183488871768234368590648637
練馬区,89,8814,578,17,694121,.000128219719616608631636270909538826803972
大田区,84,8756,438,27,666330,.000126063662149385439647021745981720768988
板橋区,93,8477,417,41,495700,.000187613475892677022392576154932418801695
江東区,72,7002,416,17,444062,.000162139521057870297390904873643770464485
足立区,88,8847,294,47,656755,.000133992127962482204170504982832258604807
杉並区,55,5390,285,17,506982,.00010848511386992043110011795290562584076
豊島区,31,3041,270,15,261282,.000118645754395633836238240674826432743166
町田市,56,5257,257,14,419062,.000133631777636722012494571209033508168242


## 地域の人口の増減に関わる要素を取得してみる
### [統計LODサポート](http://idease.info/lodstat/)で関係しそうなプロパティを片っ端から検索してみます。

- 人口：http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/  [(RDFスキーマ情報/測度)](http://trial-data.e-stat.go.jp/lodw/rdfschema/measure/#measure_population)
- sex-total: http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/sex
- age xxx :http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/age 
- 一般病院数：http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/I510120
- 地方税: http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/D320101
- 納税義務者数:http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/C120120
- 課税多少所得:http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/C120110
- 小学校数:http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/E2101
- 小学校教員数：http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/E4101
- 小学校児童数: http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/E2401
- 児童福祉施設数：http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/J2501

### データの次元を選択
[共通語彙基盤](http://imi.go.jp/ns/core/Core232.html)よりキーとなるエリアコードの次元を選択する。
これまでの例では東京都の市区町村を対象としているが、日本全域の市区町村を対象にしてみる。
まずざっくり全国の市区町村の日本人のトータルの人口を算出してみます。

In [39]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o ?ken ?siku ?areacode
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode ?areacode.
      ?areacode  ic:都道府県 ?ken;
         ic:市区町村 ?siku.
      ?s cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

o,ken,siku,areacode
7519,北海道,池田町,http://data.e-stat.go.jp/lod/sac/C01644-19700401
2960,兵庫県,三日月町,http://data.e-stat.go.jp/lod/sac/C28504-20051001
6366,岩手県,松尾村,http://data.e-stat.go.jp/lod/sac/C03306-20050901
76953,新潟県,北区,http://data.e-stat.go.jp/lod/sac/C15101-20070401
15319,北海道,倶知安町,http://data.e-stat.go.jp/lod/sac/C01400-19700401
1258,鹿児島県,里村,http://data.e-stat.go.jp/lod/sac/C46388-20041012
270897,神奈川県,戸塚区,http://data.e-stat.go.jp/lod/sac/C14110-19861103
24028,愛媛県,南宇和郡,http://data.e-stat.go.jp/lod/sac/C38500-19700401
16247,沖縄県,与那原町,http://data.e-stat.go.jp/lod/sac/C47348-19981221
2732,広島県,甲奴町,http://data.e-stat.go.jp/lod/sac/C34563-20040401


数分時間がかりました。場合によってはタイムアウトするかもしれません。